In [1]:
## This script combines osm, station and volume data

from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
import psycopg2
import findspark
findspark.init()
findspark.find()

'/home/jwang/workspace/national_energy_impact/.venv/lib/python3.8/site-packages/pyspark'

In [2]:
spark = SparkSession.builder \
        .appName("Python Spark SQL basic example") \
        .config('spark.jars', '/home/jwang/workspace/postgresql_jar/postgresql-42.6.0.jar') \
        .getOrCreate()

23/08/23 07:26:19 WARN Utils: Your hostname, jwang resolves to a loopback address: 127.0.1.1; using 192.168.2.86 instead (on interface wlp0s20f3)
23/08/23 07:26:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/08/23 07:26:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [71]:
def geo_query(pg_conn, cur, table_name, osm_table_name, schema):
     
    cur.execute(f"select count(*) from station.{table_name}")
    res1 = cur.fetchall()
    cur.execute(f"select count(*) from volume.{table_name}")
    res2 = cur.fetchall()

    if res1[0][0] > 0 and res2[0][0] > 0:  # check if table is empty
        # The station existing in volume table may not be available in the station table
        cur.execute(
        f"alter table station.{table_name} drop column if exists longitude, drop column if exists latitude;"
        f"alter table station.{table_name} add column latitude double precision, add column longitude double precision;"
        f"update station.{table_name} set latitude = \"Latitude\"::integer / 1E06, longitude = \"Longitude\"::integer / 1E06 * -1;"
        f"drop table if exists volume_station_combined_{table_name};"
        f'''CREATE TABLE if not exists volume_station_combined_{table_name} AS (SELECT distinct table_a.*, table_b.\"Num_Lanes\", table_b.\"Num_Classes\", table_b.latitude, table_b.longitude 
        FROM volume.{table_name} as table_a INNER JOIN station.{table_name} as table_b ON table_a.station_id = table_b.\"Station_Id\");'''
        f"alter table volume_station_combined_{table_name} drop column if exists geom;"
        f"alter table volume_station_combined_{table_name} add column geom geometry(point, 4326);"
        f"update volume_station_combined_{table_name} set geom = ST_Transform(ST_SetSRID(ST_MakePoint(longitude, latitude), 4326), 4326);")
        
        name_with_space = ['District of Columbia', 'North Carolina', 'North Dakota', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', \
                           'Rhode Island', 'South Carolina', 'South Dakota', 'West Virginia']

        if osm_table_name in name_with_space:
            cur.execute(f"alter table geospatial.\"{osm_table_name.lower()}\" drop column if exists location;"
                    f"alter table geospatial.\"{osm_table_name.lower()}\" add column location geometry(multilinestring, 4326);"
                    f"update geospatial.\"{osm_table_name.lower()}\" SET location = ST_GeomFromText(geometry);")
        else:
            cur.execute(f"alter table geospatial.{osm_table_name} drop column if exists location;"
                        f"alter table geospatial.{osm_table_name} add column location geometry(multilinestring, 4326);"
                        f"update geospatial.{osm_table_name} SET location = ST_GeomFromText(geometry);")

        station_list = []
        cur.execute(f'select distinct station_id from volume_station_combined_{table_name}')
        results = cur.fetchall()
        stations = list(results)

        for idx in stations:
            station_list.append(idx[0])
        
        cur.execute(f"drop table if exists {schema}.final_table_{table_name}")
        query1 = f"""
        create table if not exists {schema}.final_table_{table_name}(
        record_type text,
        state_code text,
        f_system text,
        station_id text,
        travel_dir text,
        travel_lane text,
        year_record text,
        month_record text,
        day_record text,
        day_of_week text,
        hour_00_vol text,
        hour_01_vol text,
        hour_02_vol text,
        hour_03_vol text,
        hour_04_vol text,
        hour_05_vol text,
        hour_06_vol text,
        hour_07_vol text,
        hour_08_vol text,
        hour_09_vol text,
        hour_10_vol text,
        hour_11_vol text,
        hour_12_vol text,
        hour_13_vol text,
        hour_14_vol text,
        hour_15_vol text,
        hour_16_vol text,
        hour_17_vol text,
        hour_18_vol text,
        hour_19_vol text,
        hour_20_vol text,
        hour_21_vol text,
        hour_22_vol text,
        hour_23_vol text,
        restrictions text,
        state_name text,
        num_lanes text,
        num_classes text,
        latitude double precision,
        longitude double precision,
        geom geometry,
        highway text,
        maxspeed text,
        length double precision,
        location geometry,
        distance double precision
        )"""
        cur.execute(query1)

        for s_id in station_list:
            if osm_table_name in name_with_space: 
                statement = f"""
                            with closest_segment as (
                                SELECT highway, maxspeed, length, location, ST_Distance((select geom from volume_station_combined_{table_name} where station_id = \'{s_id}\' limit 1), location) 
                                AS distance FROM geospatial.\"{osm_table_name.lower()}\" ORDER BY distance ASC LIMIT 1)
                            insert into {schema}.final_table_{table_name}
                            select * from volume_station_combined_{table_name} cross join closest_segment where station_id = \'{s_id}\';
                            """
            else:
                statement = f"""
                            with closest_segment as (
                                SELECT highway, maxspeed, length, location, ST_Distance((select geom from volume_station_combined_{table_name} where station_id = \'{s_id}\' limit 1), location) 
                                AS distance FROM geospatial.{osm_table_name} ORDER BY distance ASC LIMIT 1)
                            insert into {schema}.final_table_{table_name}
                            select * from volume_station_combined_{table_name} cross join closest_segment where station_id = \'{s_id}\';
                            """
            cur.execute(statement)
    else:
        cur.execute(f'create table {schema}.final_table_{table_name} (dummy INTEGER NOT NULL PRIMARY KEY);')

    pg_conn.commit()

In [72]:
def main():
    host = 'localhost'
    database = 'national_volume_osm'
    user = 'postgres'
    password = 'wjh00813'
    port = 5432
    conn_string = 'postgresql://{0}:{1}@{2}:{3}/{4}'.format(user, password, host, port, database)
    pg_conn = psycopg2.connect(conn_string)
    cur = pg_conn.cursor() 
    
    states_dict = {
            'AK': 'Alaska',
            'AL': 'Alabama',
            'AR': 'Arkansas',
            'AZ': 'Arizona',
            'CA': 'California',
            'CO': 'Colorado',
            'CT': 'Connecticut',
            'DC': 'District of Columbia',
            'DE': 'Delaware',
            'FL': 'Florida',
            'GA': 'Georgia',
            'HI': 'Hawaii',
            'IA': 'Iowa',
            'ID': 'Idaho',
            'IL': 'Illinois',
            'IN': 'Indiana',
            'KS': 'Kansas',
            'KY': 'Kentucky',
            'LA': 'Louisiana',
            'MA': 'Massachusetts',
            'MD': 'Maryland',
            'ME': 'Maine',
            'MI': 'Michigan',
            'MN': 'Minnesota',
            'MO': 'Missouri',
            'MS': 'Mississippi',
            'MT': 'Montana',
            'NC': 'North Carolina',
            'ND': 'North Dakota',
            'NE': 'Nebraska',
            'NH': 'New Hampshire',
            'NJ': 'New Jersey',
            'NM': 'New Mexico',
            'NV': 'Nevada',
            'NY': 'New York',
            'OH': 'Ohio',
            'OK': 'Oklahoma',
            'OR': 'Oregon',
            'PA': 'Pennsylvania',
            'RI': 'Rhode Island',
            'SC': 'South Carolina',
            'SD': 'South Dakota',
            'TN': 'Tennessee',
            'TX': 'Texas',
            'UT': 'Utah',
            'VA': 'Virginia',
            'VT': 'Vermont',
            'WA': 'Washington',
            'WI': 'Wisconsin',
            'WV': 'West Virginia',
            'WY': 'Wyoming'
    }

    year = '2022'
    table_name = list(states_dict.keys())
    osm_table_name = list(states_dict.values())
    final_schema = 'combined'
    cur.execute(f"create schema if not exists {final_schema}")

    for t1, t2 in zip(table_name, osm_table_name):
        t1 = t1 + '_' + year
        geo_query(pg_conn, cur, t1, t2, final_schema)
        print(f"{t2} is completed!")

    pg_conn.close()

In [73]:
if __name__ == "__main__":
    main()

District of Columbia is completed!
Delaware is completed!
Florida is completed!
Georgia is completed!
Hawaii is completed!
Iowa is completed!
Idaho is completed!
Illinois is completed!
Indiana is completed!
Kansas is completed!
Kentucky is completed!
Louisiana is completed!
Massachusetts is completed!
Maryland is completed!
Maine is completed!
Michigan is completed!
Minnesota is completed!
Missouri is completed!
Mississippi is completed!
Montana is completed!
North Carolina is completed!
North Dakota is completed!
Nebraska is completed!
New Hampshire is completed!
New Jersey is completed!
New Mexico is completed!
Nevada is completed!
New York is completed!
Ohio is completed!
Oklahoma is completed!
Oregon is completed!
Pennsylvania is completed!
Rhode Island is completed!
South Carolina is completed!
South Dakota is completed!
Tennessee is completed!
Texas is completed!
Utah is completed!
Virginia is completed!
Vermont is completed!
Washington is completed!
Wisconsin is completed!
West 